In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_curve, confusion_matrix, roc_auc_score

print('done')

C:\Users\lnp2\AppData\Roaming\Python\Python37\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [ ]:
%%time 

df = pd.read_csv('Intermediate_data\Trimmed.csv', 
                 parse_dates = ['earliest_cr_line', 'issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'])

df['history'] = (df.issue_d - df.earliest_cr_line).dt.days 

df.drop(['Unnamed: 0'], axis = 1, inplace =  True)

print('done')

In [ ]:
df.drop(['last_pymnt_amnt', 'collection_recovery_fee', 'recoveries', 'out_prncp_inv', 
         'out_prncp', 'total_rec_prncp', 'int_rate', 'total_pymnt', 'total_pymnt_inv',
         'total_rec_late_fee', 'total_rec_int', 'emp_title', 'debt_settlement_flag', 
         'addr_state', 'purpose', 'delinq_amnt', 'hardship_flag', 'pymnt_plan', 
         'collections_12_mths_ex_med', 'chargeoff_within_12_mths', 'acc_now_delinq', 
         'earliest_cr_line', 'issue_d', 'last_pymnt_d', 'next_pymnt_d', 
         'last_credit_pull_d'],
         axis = 1, inplace = True)

print('done')

In [ ]:
cate_list = list(df.dtypes[df.dtypes == 'object'].index)

df_cate = df[cate_list]

df_cat_dum = pd.get_dummies(df_cate, drop_first = True)
df_cat_dum.drop('loan_status_Late/Charged Off', axis = 1, inplace = True)

print('done')

In [ ]:
df_num_col = df.dtypes[df.dtypes == 'float64']

df_num = df[list(df_num_col.index)]

y = 1 -pd.get_dummies(df.loan_status, drop_first = True)

print('done')

In [ ]:
for col in df_num.columns:
    df_num[col].fillna(df_num[col].mean(), inplace = True)
    
df_com = pd.concat([df_num, df_cat_dum, y], axis = 1)

df0 = df_com[df_com['Late/Charged Off'] == 0]
df1 = df_com[df_com['Late/Charged Off'] == 1].sample(n = df0.shape[0])  

df_combined = pd.concat([df0, df1], axis = 0)
df_com = df_combined.sample(frac = 1.0)

y = df_com['Late/Charged Off']
X = df_com.drop(['Late/Charged Off'], axis = 1).values

print('done')

In [ ]:
scaled = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.5, random_state = 77)

print('done')

## Test 
model = GradientBoostingClassifier(verbose = 1,
                                       random_state  = 77)
    
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
parm_val = {}
    
fw = open("GB_FIT_detail.txt", 'a')
fw.write(str(datetime.now()) + '\t' + str(parm_val)+'\t'+ str(score)+ '\n')

fw.close()    

In [ ]:
param_grid = {'max_features': np.arange(0.05,0.5, 0.02), 'n_estimators': np.arange(5,20000,1), 
             'min_samples_leaf': np.arange(2, 10000, 5), 'min_samples_split': np.arange(1, 10000, 5),
             'max_depth': np.arange(2, 10000, 1), 'learning_rate': np.arange(0.01, 0.5, 0.05),
             'min_weight_fraction_leaf': np.arange(0,0.1, 0.02)
             }


#fw = open("GB_FIT_detail.txt","w")
#fw.write('Datetime \t parameters \t Score \n')
#fw.close

i = 0
while i < 2000:
    parm_val = dict()
    for keys, values in param_grid.items():
        parm_val[keys] = np.random.choice(values)
    
    model = GradientBoostingClassifier(max_features = parm_val['max_features'], 
                                       n_estimators = parm_val['n_estimators'],
                                       min_samples_leaf = parm_val['min_samples_leaf'],
                                       max_depth = parm_val['max_depth'],
                                       min_weight_fraction_leaf = parm_val['min_weight_fraction_leaf'], 
                                       min_samples_split = parm_val['min_samples_split'],
                                       learning_rate = parm_val['learning_rate'],
                                       random_state  = 77)
    
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    
    fw = open("GB_FIT_detail.txt", 'a')
    fw.write(str(datetime.now()) + '\t' + str(parm_val)+'\t'+ str(score)+ '\n')
    fw.close()
    
    print(datetime.now(), score)
    
    i = i+1

print('done')

2019-02-06 10:01:28.609020 0.6558384971093979


In [ ]:

read_score = pd.read_csv('GB_FIT_detail.txt', sep = '\t')

read_score.sort_values(by = ' Score ', inplace = True, ascending = False)

parm_val = eval(read_score.iloc[0,1])

model = GradientBoostingClassifier(max_features = parm_val['max_features'], 
                                   n_estimators = parm_val['n_estimators'],
                                   min_samples_leaf = parm_val['min_samples_leaf'],
                                   max_depth = parm_val['max_depth'],
                                   min_weight_fraction_leaf = parm_val['min_weight_fraction_leaf'], 
                                   min_samples_split = parm_val['min_samples_split'],
                                   learning_rate = parm_val['learning_rate'],
                                   verbose = 0,
                                   random_state = 77)

model.fit(X_train, y_train)
score = model.score(X_test, y_test)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred,  target_names = ['paid', 'not-paid']))

y_pred_prob = model.predict_proba(X_test)[:,1]

# Generate ROC curve values: fpr, tpr, thresholds
fpr,tpr, thresholds = roc_curve(y_test, y_pred_prob)

print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, 'r',label = 'Gradient Boosting')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show() 

In [ ]:
from joblib import dump, load
dump(model, 'Grad_boosting.joblib')

In [ ]:
col_list = list(df_com.drop(['Late/Charged Off'], axis = 1).columns)

feat = model.feature_importances_
feat = pd.DataFrame(data = feat, index = col_list, columns= ['Feat_imp'] )
feat.sort_values(by = 'Feat_imp', ascending = False, inplace = True)
feat.to_csv('Intermediate_data\Feature_imp_GB.csv')

print('done')